In [1]:
import requests
import pandas as pd

req = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/records?")
wb = req.json()

url_belibstat = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/records?order_by=date_maj%20DESC"

url_belibstat_json = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/exports/json?order_by=date_maj%20DESC'

url_count = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/records?"

url_count_json = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/exports/json?order_by=k&sort=desc&limit=10000"


url_belibdyn = "https://parisdata.opendatasoft.com/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-disponibilite-temps-reel/records?order_by=last_updated%20DESC&limit=100"

In [2]:
print(req.content[:1000])



b'{"total_count": 1947, "results": [{"id_pdc_local": "FR*V75*E9015*06*2", "statut_pdc": "En service", "arrondissement": "15e Arrondissement", "id_station_local": "FR*V75*P9015*06", "id_station_itinerance": "FRV75P901506", "nom_station": "Paris | Rue Desnouettes 55", "code_insee_commune": "75115", "implantation_station": "Voirie", "nbre_pdc": "3", "date_maj": "2024-07-01", "condition_acces": "Acc\\u00e8s libre", "adresse_station": "55 Rue Desnouettes 75015 Paris", "coordonneesxy": {"lon": 2.286912, "lat": 48.83571}, "gratuit": "false", "paiement_acte": "true", "paiement_cb": "false", "paiement_autre": "true", "reservation": "false", "observations": ["Paiement autre - badge RFID Belib ou autre, application Belib, QRCode sur chaque PDC, site internet Belib", "Plus d\'information - https://belib.paris.fr"], "num_pdl": "07436179348549", "siren_amenageur": "531680445", "contact_amenageur": "assistance.technique@tevgo.fr", "nom_amenageur": "TOTALENERGIES", "nom_operateur": "TOTALENERGIES", "c

In [3]:
import pandas as pd
import json

def get_dpe_from_url(url):
    req = requests.get(url)
    
    # Décoder la réponse en bytes vers une chaîne
    decoded_content = req.content.decode("utf-8")
    
    # Charger le JSON à partir de la chaîne décodée
    wb = json.loads(decoded_content)
    # Vérifier si la clé 'results' existe
    if isinstance(wb, list):
            # Directly normalize the list of records into a DataFrame
            df = pd.DataFrame(wb)
    else:
            # If the response contains a 'results' key, normalize that
            df = pd.json_normalize(wb.get("results"))
    
    return df

def get_dpe2_from_url(url):
    req = requests.get(url)
    
    # Décoder la réponse en bytes vers une chaîne
    decoded_content = req.content.decode("utf-8")
    
    # Charger le JSON à partir de la chaîne décodée
    wb = json.loads(decoded_content)
    # Vérifier si la clé 'results' existe
    if isinstance(wb, list):
            # Directly normalize the list of records into a DataFrame
            df = pd.DataFrame(wb)
    else:
            # If the response contains a 'results' key, normalize that
            df = pd.json_normalize(wb.get("results"))

    df_filtered = df[~df['libelle'].str.startswith("PI") & 
    ~df['libelle'].str.startswith("PE") & 
    ~df['libelle'].str.startswith("A3") & 
    ~df['libelle'].str.startswith("AE") & 
    ~df['libelle'].str.startswith("AI")& 
    ~df['libelle'].str.startswith("A6")& 
    ~df['libelle'].str.startswith("SI")& 
    ~df['libelle'].str.startswith("SE")]

    df_cleaned = df_filtered.drop_duplicates(subset=['libelle'])
    
    return df_cleaned

def get_df_from_url(url):
    # Send a GET request to the API URL
    req = requests.get(url)
    
    # Ensure the request was successful (status code 200)
    if req.status_code == 200:
        wb = req.json()  # Parse the JSON response
        
        # Check if the response is directly a list of records
        if isinstance(wb, list):
            # Directly normalize the list of records into a DataFrame
            df = pd.json_normalize(wb)
        else:
            # If the response contains a 'results' key, normalize that
            df = pd.json_normalize(wb.get("results"))
        
        df_en_service = df[df['statut_pdc'] == 'En service']

        # Trier les données filtrées par 'date_maj' en ordre décroissant (ou croissant selon votre besoin)
       
        df_cleaned = df_en_service.drop_duplicates(subset=['id_pdc_local'])

       
        
        return df_cleaned
    else:
        # Handle failed request (non-200 status code)
        print(f"Failed to retrieve data: {req.status_code}")
        return None


In [4]:
dpe_belibstat = get_df_from_url(url_belibstat_json)
dpe_count = get_dpe_from_url(url_count)
dpe_belibdyn = get_dpe_from_url(url_belibdyn)
dpe_count_json = get_dpe2_from_url(url_count_json)



In [5]:

dpe_belibstat.head(2)



,id_pdc_local,statut_pdc,arrondissement,id_station_local,id_station_itinerance,nom_station,code_insee_commune,implantation_station,nbre_pdc,date_maj,...,prise_type_ef,prise_type_2,prise_type_combo_ccs,prise_type_chademo,prise_type_autre,prise_type_3,horaires,raccordement,coordonneesxy.lon,coordonneesxy.lat
0,FR*V75*EPX05*14*4,En service,05e Arrondissement,FR*V75*BPX05*14,FRV75BPX0514,Paris | Rue Claude Bernard 36,75105,Voirie,6,2024-10-10,...,True,True,False,False,False,False,24/7,Direct,2.348008,48.839592
1,FR*V75*EPX05*14*1,En service,05e Arrondissement,FR*V75*BPX05*14,FRV75BPX0514,Paris | Rue Claude Bernard 36,75105,Voirie,6,2024-10-10,...,True,True,False,False,False,False,24/7,Direct,2.348008,48.839592


In [6]:
dpe_belibdyn.head(2)

,id_pdc,statut_pdc,url_description_pdc,last_updated,adresse_station,code_insee_commune,arrondissement,coordonneesxy.lon,coordonneesxy.lat,coordonneesxy
0,FR*V75*EPX07*06*5,Disponible,https://parisdata.opendatasoft.com/explore/dat...,2024-12-10T08:02:09+00:00,5 Rue de Bellechasse 75007 Paris,75107,07e Arrondissement,2.324505,48.85988,NaN
1,FR*V75*E9016*02*1,Occupé (en charge),https://parisdata.opendatasoft.com/explore/dat...,2024-12-10T08:02:06+00:00,37 Avenue d'Iéna 75016 Paris,75116,16e Arrondissement,2.295469,48.86851,NaN


In [7]:
dpe_count_json.head(2)

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d,geo_shape
0,271,VGP_Cours_la_Reine,2023-11-06T08:00:00+00:00,853.0,0.0,Fluide,163,Souterrain_Cours_la_Reine,135,Cours_la_Reine-Concorde,Invalide,1996-10-03,2023-01-01,"{'lon': 2.3084567115959773, 'lat': 48.86467042...","{'type': 'Feature', 'geometry': {'coordinates'..."
1,4624,Souterrain_Alma_VGP,2023-11-06T04:00:00+00:00,NaN,0.0,Fluide,2447,Av_de_New_York-Debrousse,2467,Cours_Albert_Ier-VGP,Invalide,1996-10-16,2023-01-01,"{'lon': 2.301553929792855, 'lat': 48.864264855...","{'type': 'Feature', 'geometry': {'coordinates'..."


# Affichage carte 

In [8]:
import geopandas as gpd

def get_geo_from_dpe(df):

    # Supprimer les lignes où les coordonnées sont manquantes
    df = df.dropna(subset=["coordonneesxy.lon", "coordonneesxy.lat"])

    # Créer un GeoDataFrame avec des points géographiques
    dpe = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["coordonneesxy.lon"], df["coordonneesxy.lat"]),crs=4326
          
    )

    return dpe



In [9]:
geo_belibdyn = get_geo_from_dpe(dpe_belibdyn)
geo_belibstat = get_geo_from_dpe(dpe_belibstat)


geo_belibdyn.head(2)

In [10]:
geo_belibdyn.head(2)

,id_pdc,statut_pdc,url_description_pdc,last_updated,adresse_station,code_insee_commune,arrondissement,coordonneesxy.lon,coordonneesxy.lat,coordonneesxy,geometry
0,FR*V75*EPX07*06*5,Disponible,https://parisdata.opendatasoft.com/explore/dat...,2024-12-10T08:02:09+00:00,5 Rue de Bellechasse 75007 Paris,75107,07e Arrondissement,2.324505,48.85988,NaN,POINT (2.3245 48.85988)
1,FR*V75*E9016*02*1,Occupé (en charge),https://parisdata.opendatasoft.com/explore/dat...,2024-12-10T08:02:06+00:00,37 Avenue d'Iéna 75016 Paris,75116,16e Arrondissement,2.295469,48.86851,NaN,POINT (2.29547 48.86851)


In [11]:
import seaborn as sns
import folium
import pandas as pd
from matplotlib.colors import to_hex

# Palette de couleurs
palette = sns.color_palette("tab10", 10)  # 10 couleurs distinctes

def interactive_map_dpe(dpe):
    


    # Supprimer les lignes avec des NaN dans coordonneesxy.lat ou coordonneesxy.lon
    dpe_clean = dpe.dropna(subset=["coordonneesxy.lat", "coordonneesxy.lon"])


    # Associer une couleur unique à chaque statut
    unique_statuses = dpe_clean["statut_pdc"].unique()
    status_colors = {status: to_hex(palette[i % len(palette)]) for i, status in enumerate(unique_statuses)}

    # Ajouter la colonne de couleur basée sur statut_pdc
    dpe_clean["color"] = dpe_clean["statut_pdc"].map(status_colors)

    # Calculer le centre de la carte et ses limites
    center = dpe_clean[["coordonneesxy.lat", "coordonneesxy.lon"]].mean().values.tolist()
    sw = dpe_clean[["coordonneesxy.lat", "coordonneesxy.lon"]].min().values.tolist()
    ne = dpe_clean[["coordonneesxy.lat", "coordonneesxy.lon"]].max().values.tolist()

    print(f"Centre de la carte : {center}")  # Afficher le centre pour vérifier

    # Initialiser la carte Folium
    m = folium.Map(location=center, tiles="OpenStreetMap")

    # Ajouter les marqueurs à la carte
    for i in range(len(dpe_clean)):
        folium.Marker(
            [dpe_clean.iloc[i]["coordonneesxy.lat"], dpe_clean.iloc[i]["coordonneesxy.lon"]],
            popup=f"Adresse: {dpe_clean.iloc[i]['adresse_station']}, <br>"
                  f"Arrondissement: {dpe_clean.iloc[i]['arrondissement']}, <br>"
                  f"date: {dpe_clean.iloc[i]['last_updated']}, <br>"
                  f"Statut: {dpe_clean.iloc[i]['statut_pdc']}",
            icon=folium.Icon(
                color="black", icon="info-sign", icon_color=dpe_clean.iloc[i]["color"]
            ),
        ).add_to(m)

    # Ajuster les limites de la carte
    m.fit_bounds([sw, ne])

    return m

# Appel de la fonction avec vos données
map_belibdyn = interactive_map_dpe(geo_belibdyn)





Centre de la carte : [48.86151490816326, 2.3279267642857144]


Affichage de la carte enlever le diez pour l'afficher

In [12]:
#map_belibdyn

Affichage de la carte statique

In [13]:


from folium.plugins import MarkerCluster
from shapely.geometry import Point
import pandas as pd

def create_interactive_map_with_density(gdf, radius=500):
    """
    Crée une carte interactive où la taille des marqueurs est ajustée
    en fonction de la densité des stations dans un rayon donné (en mètres).
    """
    # Convertir les données de stations en GeoDataFrame
    
    # Créer une carte centrée sur la moyenne des coordonnées
    center = gdf[["coordonneesxy.lat", "coordonneesxy.lon"]].mean().values.tolist()
    sw = gdf[["coordonneesxy.lat", "coordonneesxy.lon"]].min().values.tolist()
    ne = gdf[["coordonneesxy.lat", "coordonneesxy.lon"]].max().values.tolist()
    m = folium.Map(location=center, zoom_start=12, tiles="OpenStreetMap")
    
    # Créer un groupe de marqueurs avec MarkerCluster pour gérer les clusters
    marker_cluster = MarkerCluster().add_to(m)

    # Ajouter les marqueurs pour chaque station
    for idx, row in gdf.iterrows():
        lat = row['coordonneesxy.lat']
        lon = row['coordonneesxy.lon']
        
        # Calculer le nombre de stations dans un rayon donné autour de la station actuelle
        point = Point(lon, lat)
        buffer = point.buffer(radius / 111000)  # 1 degré = environ 111 km, donc pour des mètres, on divise par 111000
        nearby_stations = gdf[gdf.geometry.within(buffer)]
        
        # Ajuster la taille du marqueur en fonction du nombre de stations proches
        num_nearby = len(nearby_stations)
        color = "blue" if num_nearby < 5 else "green" if num_nearby < 10 else "red"
        size = min(10 + num_nearby, 30)  # Taille des marqueurs allant de 10 à 30 selon la densité

        folium.CircleMarker(
            location=[lat, lon],
            radius=size,  # Ajuster la taille en fonction du nombre de stations proches
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"Station: {row['adresse_station']}<br>Arrondissement: {row['arrondissement']}<br>Stations voisines: {num_nearby}"
        ).add_to(marker_cluster)

    return m

# Exemple de DataFrame avec des stations



# Créer la carte interactive
m = create_interactive_map_with_density(geo_belibstat, radius=500)




In [14]:
#m

Partie 3 chercher la lattitude et longitude des rues fréquentées


In [15]:
dpe_count_json.head(2)

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d,geo_shape
0,271,VGP_Cours_la_Reine,2023-11-06T08:00:00+00:00,853.0,0.0,Fluide,163,Souterrain_Cours_la_Reine,135,Cours_la_Reine-Concorde,Invalide,1996-10-03,2023-01-01,"{'lon': 2.3084567115959773, 'lat': 48.86467042...","{'type': 'Feature', 'geometry': {'coordinates'..."
1,4624,Souterrain_Alma_VGP,2023-11-06T04:00:00+00:00,NaN,0.0,Fluide,2447,Av_de_New_York-Debrousse,2467,Cours_Albert_Ier-VGP,Invalide,1996-10-16,2023-01-01,"{'lon': 2.301553929792855, 'lat': 48.864264855...","{'type': 'Feature', 'geometry': {'coordinates'..."


In [17]:
import geopandas as gpd

def get_geo_from_dpe_count(df):
    # Vérifiez si la colonne 'geo_point_2d' existe
    if "geo_point_2d" not in df.columns:
        raise ValueError("La colonne 'geo_point_2d' doit exister dans le DataFrame.")
    
    # Extraire longitude et latitude à partir des dictionnaires
    df["longitude"] = df["geo_point_2d"].apply(lambda x: x.get("lon") if x else None)
    df["latitude"] = df["geo_point_2d"].apply(lambda x: x.get("lat")if x else None)
    
    # Supprimer les lignes où les coordonnées sont manquantes
    df = df.dropna(subset=["longitude", "latitude"])
    
    # Créer un GeoDataFrame avec des points géographiques
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
        crs="EPSG:4326"  # Utilise WGS84 comme système de référence
    )
    
    return gdf

# Exemple d'appel
geo_count_j = get_geo_from_dpe_count(dpe_count_json)


In [18]:
geo_count_j.head(2)

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d,geo_shape,longitude,latitude,geometry
0,271,VGP_Cours_la_Reine,2023-11-06T08:00:00+00:00,853.0,0.0,Fluide,163,Souterrain_Cours_la_Reine,135,Cours_la_Reine-Concorde,Invalide,1996-10-03,2023-01-01,"{'lon': 2.3084567115959773, 'lat': 48.86467042...","{'type': 'Feature', 'geometry': {'coordinates'...",2.308457,48.864670,POINT (2.30846 48.86467)
1,4624,Souterrain_Alma_VGP,2023-11-06T04:00:00+00:00,NaN,0.0,Fluide,2447,Av_de_New_York-Debrousse,2467,Cours_Albert_Ier-VGP,Invalide,1996-10-16,2023-01-01,"{'lon': 2.301553929792855, 'lat': 48.864264855...","{'type': 'Feature', 'geometry': {'coordinates'...",2.301554,48.864265,POINT (2.30155 48.86426)


In [19]:


from folium.plugins import MarkerCluster
from shapely.geometry import Point
import pandas as pd

def create_interactive_map_with_density(gdf, circ, radius=500):
    """
    Crée une carte interactive où la taille des marqueurs est ajustée
    en fonction de la densité des stations dans un rayon donné (en mètres).
    """
    # Convertir les données de stations en GeoDataFrame
    
    # Créer une carte centrée sur la moyenne des coordonnées
    center = circ[["latitude", "longitude"]].mean().values.tolist()
    sw = circ[["latitude", "longitude"]].min().values.tolist()
    ne = circ[["latitude", "longitude"]].max().values.tolist()
    m = folium.Map(location=center, zoom_start=12, tiles="OpenStreetMap")
    
    # Créer un groupe de marqueurs avec MarkerCluster pour gérer les clusters
    marker_cluster = MarkerCluster().add_to(m)

    # Ajouter les marqueurs pour chaque station
    for idx, row in circ.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        
        # Calculer le nombre de stations dans un rayon donné autour de la station actuelle
        point = Point(lon, lat)
        buffer = point.buffer(radius / 111000)  # 1 degré = environ 111 km, donc pour des mètres, on divise par 111000
        nearby_stations = gdf[gdf.geometry.within(buffer)]

        k = row.get("k", 0)  # Par défaut, si `taux_occupation` est absent, k = 0
        
        
        # Ajuster la taille du marqueur en fonction du nombre de stations proches
        num_nearby = 1 + len(nearby_stations) # Si il n'y apas de station juste la densité compte sinon on divise par le nombre de station 
        indisponibility = k/num_nearby
        color = "blue" if indisponibility < 5 else "green" if indisponibility < 10 else "red"
        size = min(10 + indisponibility, 30)  # Taille des marqueurs allant de 10 à 30 selon la densité

        folium.CircleMarker(
            location=[lat, lon],
            radius=size,  # Ajuster la taille en fonction du nombre de stations proches
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"Adresse: {row['libelle']}<br>Indisponibilité: {indisponibility}<br>trafic: {row['etat_trafic']}<br>Nombre de station: {num_nearby}<br>Débit: {row['q']}"
        ).add_to(marker_cluster)

    return m

# Exemple de DataFrame avec des stations



# Créer la carte interactive
m = create_interactive_map_with_density(geo_belibstat,geo_count_j, radius=500)




In [119]:
m

On remarque que ça marche mais la majorité des places qu'on devrait avoir serait au niveau des autoroutes ou des périphériques

Nouvel url sans autoroutes et périphériques intérieurs et extérieurs